In [ ]:
import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import google.generativeai as genai

# File paths
CREDENTIALS_FILE = " "  # Your credentials file
TOKEN_FILE = " "#enter your tokrn file link

# Google Gemini API Key
GEMINI_API_KEY = " "#enter your llm appi key

# Define scopes for sending and reading emails
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

def authenticate():
    """Authenticate and return the Gmail API service."""
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

def decode_base64(data):
    """Helper function to decode Base64 email content safely."""
    try:
        return base64.urlsafe_b64decode(data).decode()
    except Exception:
        return "(Unable to decode email content)"

def extract_email_body(payload):
    """Extracts text body from email payload, handling attachments."""
    if "parts" in payload:
        for part in payload["parts"]:
            mime_type = part.get("mimeType", "")
            if mime_type == "text/plain":
                return decode_base64(part["body"]["data"])
            elif mime_type == "multipart/alternative":
                for sub_part in part.get("parts", []):
                    if sub_part.get("mimeType") == "text/plain":
                        return decode_base64(sub_part["body"]["data"])
    elif "body" in payload and "data" in payload["body"]:
        return decode_base64(payload["body"]["data"])
    
    return "(No text content found)"

def fetch_latest_email(service):
    """Fetch the latest email, extract text, and detect attachments."""
    try:
        response = service.users().messages().list(userId="me", maxResults=1).execute()
        messages = response.get("messages", [])

        if not messages:
            print(" No new emails found.")
            return None, None, None

        message_id = messages[0]["id"]
        email = service.users().messages().get(userId="me", id=message_id, format="full").execute()

        headers = email["payload"]["headers"]
        subject = next((h["value"] for h in headers if h["name"] == "Subject"), "No Subject")
        sender = next((h["value"] for h in headers if h["name"] == "From"), "Unknown Sender")

        body = extract_email_body(email["payload"])
        
        print(f"\n📩 **New Email Received**")
        print(f"📧 **From:** {sender}")
        print(f"📜 **Subject:** {subject}")
        print(f"📖 **Message:** {body.strip()}")
        
        return sender, subject, body.strip()
    except Exception as e:
        print(f"❌ Error retrieving email: {e}")
        return None, None, None

def generate_gemini_reply(message_text):
    """Generate a reply using Google Gemini AI, making it sound like the user is responding personally."""
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel()
        
        response = model.generate_content(
            f"Write a natural, professional, and human-like reply to this email as if I am responding personally. "
            f"Do NOT include any names or greetings—just the response itself:\n\n{message_text}"
        )
        
        return response.text.strip() if response else "No response generated."
    except Exception as e:
        print(f"❌ Failed to generate AI response: {e}")
        return None




def send_email(service, to_email, subject, message_text, attachment_path=None):
    """Send an email with an optional attachment."""
    try:
        message = MIMEMultipart()
        message["to"] = to_email
        message["subject"] = subject

        message.attach(MIMEText(message_text, "plain"))

        # Attach file if provided
        if attachment_path:
            try:
                with open(attachment_path, "rb") as file:
                    part = MIMEBase("application", "octet-stream")
                    part.set_payload(file.read())
                
                encoders.encode_base64(part)
                part.add_header("Content-Disposition", f"attachment; filename={os.path.basename(attachment_path)}")
                message.attach(part)
                print("📎 Attachment added successfully.")
            except Exception as e:
                print(f"❌ Failed to attach file: {e}")

        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        send_request = service.users().messages().send(userId="me", body={"raw": raw_message}).execute()

        print("✅ Email sent successfully!")
    
    except Exception as e:
        print(f"❌ Error sending email: {e}")

def ask_for_attachment():
    """Ask user if they want to attach a file and return the file path."""
    attach_file = None  
    attach_input = input("\n📎 Do you want to attach a file? (yes/no): ").strip().lower()
    if attach_input == "yes":
        attach_file = input("📂 Enter the file path: ").strip()
        if not os.path.exists(attach_file):
            print("❌ Invalid file path! Sending email without an attachment.")
            attach_file = None
    return attach_file

if __name__ == "__main__":
    service = authenticate()
    sender, subject, message_text = fetch_latest_email(service)
    
    if sender and message_text:
        ai_reply = generate_gemini_reply(message_text)
        
        if ai_reply:
            print(f"\n💬 **AI Generated Reply:** {ai_reply}")
            user_input = input("\n📩 Do you want to send this reply? (yes/no): ").strip().lower()
            
            if user_input == "yes":
                attachment_path = ask_for_attachment()
                send_email(service, sender, f"Re: {subject}", ai_reply, attachment_path)

    # Ask if the user wants to send another email block
    while True:
        another_email = input("\n📧 Do you want to send another email to a different recipient? (yes/no): ").strip().lower()
        if another_email == "yes":
            recipient_email = input("📨 Enter the recipient's email address: ").strip()
            new_subject = input("✉️ Enter the subject of the email: ").strip()
            new_message = input("📝 Enter your message: ").strip()

            attachment_path = ask_for_attachment()
            send_email(service, recipient_email, new_subject, new_message, attachment_path)
        else:
            print("\n🚪 Exiting program. Have a great day! 👋")
            break
